**오늘은 해당 평가 지표 중, MSE, RMSE, MAE, $R^2$ 값을 Python으로 산출하는 방법을 알아볼 것입니다.**

데이터를 로드하여 간단한 전처리를 수행하고, 다중회귀모델을 모델링한 뒤 선형회귀모델 평가지표를 통해 모델을 평가해보겠습니다.

### **선형회귀모델 평가지표**

선형회귀모델에서는 다양한 모델 평가지표가 존재합니다.

* **MAE(Mean Absolute Error)** : 실제값과 예측값의 차이를 절댓값으로 변환해 평균한 것
* **MSE(Mean Squared Error)** : 실제값과 예측값의 차이를 제곱해 평균한 것
* **MSLE** : MSE에 로그를 적용한 것. 결정값이 클 수록 오류값이 커져서, 일부 큰 오류값 때문에 전체 오류값이 커지는 것을 막음
* **RMSE** : MSE 값은 오류의 제곱이라 실제 오류평균보다 더 커질 수 있으므로, MSE에 루트를 씌운 것.
* **RMSLE** : RMSE에 로그를 적용한 것. 결정값이 클 수록 오류값이 커져서, 일부 큰 오류값 때문에 전체 오류값이 커지는 것을 막음
* **R^2** : 분산 기반으로 예측 성능을 평가. 실제 값의 분산 대비 예측값의 분산 비율을 지표로 함 (예측값/실제값). 1에 가까울수록 예측 정확도가 높음.

### **1. 데이터 Load, 간단한 전처리 수행**

In [ ]:
#필요한 lib Import, 데이터 Load

from google.colab import files
import io
import pandas as pd

uploaded = files.upload()

Saving kc_house_data.csv to kc_house_data.csv


In [ ]:
# 데이터셋 Load
df = pd.read_csv(io.BytesIO(uploaded['kc_house_data.csv']))
df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [ ]:
# 전처리 1. 'date' feature에서 T를 기준으로 데이터 셋을 나눠, 정확한 날짜값 산출 
df['date'] = pd.to_numeric(df['date'].str.replace('T','').str[:8])

In [ ]:
# 전처리 2. 날짜값 기준으로 데이터 정렬
df['date'] = df['date'].sort_values(ascending=True).reset_index(drop=True)

In [ ]:
# 전처리 3. Test값과 Train값을 2015년 01월 01일 기준으로 분할

y_test = df[df['date']>20150101]['price']
y_train = df[df['date']<=20150101]['price']
X_test = df[df['date']>20150101].drop(['price'],axis=1)
X_train = df[df['date']<=20150101].drop(['price'],axis=1)

### **2. 특성공학(feature engineering) 진행**

* 값이 -1.0 ~ -0.7 이면, 강한 음적 상관관계
* 값이 -0.7 ~ -0.3 이면, 뚜렷한 음적 상관관계
* 값이 -0.3 ~ -0.1 이면, 약한 음적 상관관계
* 값이 -0.1 ~ +0.1 이면, 없다고 할 수 있는 상관관계
* 값이 +0.1 ~ +0.3 이면, 약한 양적 상관관계
* 값이 +0.3 ~ +0.7 이면, 뚜렷한 양적 상관관계
* 값이 +0.7 ~ +1.0 이면, 강한 양적 상관관계


In [ ]:
corr = df.corr(method = 'pearson')
corr_result = pd.DataFrame(corr['price'])

# corr이 0.3보다 크면 뚜렷한 양의 상관관계가 존재하는 것이므로, 이를 기준으로 특성공학을 진행합니다.
corr_columns=corr_result[corr_result['price'] >= 0.3].drop(['price']).index.tolist()

In [ ]:
#뚜렷한 양의 상관관계를 보이는 features를 확인합니다.
corr_columns

['bedrooms',
 'bathrooms',
 'sqft_living',
 'view',
 'grade',
 'sqft_above',
 'sqft_basement',
 'lat',
 'sqft_living15']

### **3. 다중선형회귀 모델 생성**

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# 모델 fit
model.fit(X_train, y_train)
y_pred = model.predict(X_train)

# 예측값 확인
y_pred

array([204280.55399057, 732802.82760465, 395807.16414042, ...,
       399114.98994392, 536078.39790104, 322721.45635232])

### **4. 모델의 회귀계수(coefficients)와 절편(intercept) 확인**

In [ ]:
model.intercept_, model.coef_

(-66786315.60487911,
 array([-6.58275664e-07,  3.90758610e+00, -3.84466974e+04,  3.85355570e+04,
         1.15414777e+02,  1.48637352e-01, -1.95543487e+03,  6.45720983e+05,
         5.02768776e+04,  2.53212494e+04,  8.94962494e+04,  7.74250824e+01,
         3.79896951e+01, -2.92121264e+03,  2.19243322e+01, -5.82703817e+02,
         5.80257186e+05, -1.86123374e+05,  3.07014515e+01, -4.60045630e-01]))

### **5. MSE, RMSE, MAE, $R^2$ 를 훈련/테스트 데이터에 대해 확인**

In [ ]:
y = y_train.iloc[0:len(y_pred)]

In [ ]:
from sklearn.metrics import  mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(y, y_pred)
mae = mean_absolute_error(y, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y, y_pred)

In [ ]:
print(mse,rmse,mae,r2)

40663131336.98907 201651.01372665862 124805.33638742082 0.699040791194079
